# WORD VECTORS FEATURES

In [14]:
import numpy as np
import pandas as pd
import os
import re
import tqdm

In [11]:
os.chdir(r'C:\Users\C336682\Documents\fake\IC_FakeNews-master\\')

In [5]:
data = pd.read_csv('news_concat_01.csv').drop(columns=['Unnamed: 0'])

In [8]:
data.head()

,Label,Title
0,1,Indústria brasileira reage com melhora do comé...
1,1,A bancarrota de Detroit deixa no ar as pensões...
2,1,"PIB no Brasil cai 0,5% na leitura trimestral, ..."
3,1,O órgão supervisor europeu questiona o trabalh...
4,1,"Vega Sicilia, a ilusão da escassez"


In [97]:
from gensim.models import KeyedVectors
from nltk import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\C336682\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [28]:
model = KeyedVectors.load_word2vec_format('embeddings\w2v_cbow_s300\cbow_s300.txt', unicode_errors="ignore")

In [89]:
model.vocab['MAGIA']

In [98]:
def sent2vec(s):
    words = str(s).upper()
    words = word_tokenize(words)
    #words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(model[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v ** 2).sum())

In [100]:
sent2vec('A magia está no ar')

array([ 8.26025903e-02,  7.24097574e-03, -7.38379126e-03, -5.79546094e-02,
       -9.62195545e-02, -1.51402904e-02,  8.44508596e-03, -8.75014663e-02,
        1.46775153e-02,  8.53199959e-02, -2.42008027e-02, -2.63336226e-02,
       -7.82743841e-03, -2.42232718e-03, -2.04831883e-02,  5.11138141e-02,
       -1.17282696e-01, -8.91963914e-02, -4.45625409e-02, -5.22044599e-02,
       -2.02399176e-02,  2.33970731e-02, -6.84385449e-02, -2.85574216e-02,
        4.31255586e-02, -7.94489607e-02, -4.85189706e-02, -8.65993127e-02,
       -1.24169901e-01,  4.16343585e-02, -7.01472163e-02,  1.28226988e-02,
        4.99262400e-02,  8.42963979e-02, -1.11995317e-01, -2.83923093e-03,
        3.37016326e-03, -9.19047073e-02, -6.21290617e-02, -5.73009392e-03,
       -8.47000163e-04,  8.76494721e-02, -8.53980053e-03, -3.16273049e-02,
       -1.90150104e-02, -7.78398244e-03,  1.28775179e-01,  7.17182755e-02,
        3.86259407e-02,  1.09461593e-02,  4.65973094e-02, -3.09485588e-02,
        1.46812266e-02,  

In [109]:
X_w2v_cbow_list = [sent2vec(x) for x in data['Title']]

In [111]:
X_w2v_cbow = pd.DataFrame()

for j in range(len(X_w2v_cbow_list)):
    X_w2v_cbow[j] = X_w2v_cbow_list[j]

In [112]:
X_w2v_cbow = X_w2v_cbow.T

In [114]:
columns = []
for j in range(1,301):
    columns.append('feature'+str(j))

In [115]:
X_w2v_cbow.columns = columns

In [116]:
X_w2v_cbow.head()

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature291,feature292,feature293,feature294,feature295,feature296,feature297,feature298,feature299,feature300
0,0.042570,0.012104,0.074938,-0.013384,0.092475,-0.031281,-0.070357,-0.024914,-0.004816,0.013637,...,0.005758,-0.077279,-0.017853,-0.006452,-0.017334,-0.106524,-0.065605,0.046967,-0.037649,-0.061136
1,-0.018268,0.116134,0.088298,-0.064570,0.011260,0.039332,0.083535,0.042699,-0.000626,0.071059,...,-0.057831,-0.023792,0.017571,-0.036500,-0.066888,-0.001920,0.123694,-0.045279,0.013808,-0.082070
2,0.095050,0.034736,-0.040186,-0.048977,0.070348,0.006896,0.065039,0.089740,-0.068805,-0.027196,...,0.003375,0.035696,0.043836,-0.121432,-0.002649,0.018771,0.087685,0.047486,-0.124016,-0.026470
3,0.019449,0.099607,-0.023152,0.035851,-0.018510,0.036798,0.032469,0.067076,0.024612,0.009722,...,0.063799,-0.092861,0.096599,-0.058167,-0.003396,0.069895,0.141475,-0.025738,0.007802,-0.052845
4,0.064135,-0.029280,-0.068144,-0.130111,0.004120,-0.007988,0.058953,0.000371,0.019016,0.033978,...,-0.002356,0.042550,0.035945,0.034265,-0.035605,0.000997,-0.022376,-0.005851,-0.010347,0.033598


In [121]:
import xgboost as xgb
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [122]:
label_encoder = preprocessing.LabelEncoder()
y = label_encoder.fit_transform(data.Label.values)
X = X_w2v_cbow.values

Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, y,
                                            random_state = 42,
                                            test_size = 0.2,
                                            stratify = y) 

In [126]:
clf = xgb.XGBClassifier(nthread=10, silent=False)
clf.fit(Xtrain, Ytrain)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=10, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=False, subsample=1)

In [127]:
clf.score(Xtest, Ytest)

C:\Users\C336682\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.91495093323071

In [128]:
news = ["Em entrevista, Moro fala em reação, cita risco de morte e dá duríssimo recado aos bandidos so Brasil",
        "Bolsonaro ganha em disparada na contagem de votos do IBGE",
        "Lula condenado à 10 anos de cadeia pelo caso do Triplex",
        "Lula ordena a prisão de Sérgio Moro",
        "Peixe alienígena encontrado no Rio de Janeiro",
        "Lula preso amanhã",
        "Terremoto no Brasil destroi cidades no Alagoas",
        "Dilma reassumiu a presidência após prisão de Michel Temer",
        "Michel Temer assume presidência do Brasil",
        "Toffoli propõe estender restrição ao foro para todas as autoridades",
        "Intenção de votos em Bolsonaro dispara no mês de Outubro"]

In [182]:
def classify(noticia):
    
    test=sent2vec(noticia)
    vector = np.zeros((1,300))
    
    j=0
    for n in test:
        vector[0,j] = n
        j+=1
    

    prob = round((clf.predict_proba(vector)[0][0]),4)*100
    print("A notícia tem " + str(prob)[:5] + "% de probalidade de ser falsa.")


In [183]:
for n in news:
    print(n)
    classify(n)
    print('\n')

Em entrevista, Moro fala em reação, cita risco de morte e dá duríssimo recado aos bandidos so Brasil
A notícia tem 96.18% de probalidade de ser falsa.


Bolsonaro ganha em disparada na contagem de votos do IBGE
A notícia tem 60.22% de probalidade de ser falsa.


Lula condenado à 10 anos de cadeia pelo caso do Triplex
A notícia tem 64.09% de probalidade de ser falsa.


Lula ordena a prisão de Sérgio Moro
A notícia tem 95.78% de probalidade de ser falsa.


Peixe alienígena encontrado no Rio de Janeiro
A notícia tem 28.04% de probalidade de ser falsa.


Lula preso amanhã
A notícia tem 50.16% de probalidade de ser falsa.


Terremoto no Brasil destroi cidades no Alagoas
A notícia tem 20.56% de probalidade de ser falsa.


Dilma reassumiu a presidência após prisão de Michel Temer
A notícia tem 52.43% de probalidade de ser falsa.


Michel Temer assume presidência do Brasil
A notícia tem 35.80% de probalidade de ser falsa.


Toffoli propõe estender restrição ao foro para todas as autoridades
A 